In [1]:
def get_dataset(path):
    dataset = [] # the entire dataset
    import json
    
    # Opening JSON file
    f = open(path)

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Closing file
    f.close()
    content = data['data']
    n_traces = len(content)
    print("The number of traces is: ",n_traces)
    avg_shortest_paths = []
    avg_clustering = []
    diameters = []
    avg_entities = []
    avg_relations = []
    for elem in content:
        first = elem
        second = first['spans']

        triplets = [] #head, link, tail ---> process(spanID), operationName, process(references_spanID)

        dict_span_process = {}
        for elem in second:
            dict_span_process[elem['spanID']] = first['processes'][elem['processID']]['serviceName']

        root = first['traceID']

        for elem in second:
            head = elem['spanID']
            link = elem['operationName']
            tail = None
            if head != root:
                tail = elem['references'][0]['spanID']
                triplets.append([dict_span_process[head],link,dict_span_process[tail]])


        dataset.append(triplets) #adding a single graph instance to the dataset
        
    
    return dataset


In [2]:
import os
mypath= os.getcwd() + '/social-network/sample-rate-20/social-graph-service.json' 
dataset = get_dataset(mypath)

The number of traces is:  7715


In [3]:
dataset

[[['nginx-web-server', 'Follow', 'nginx-web-server'],
  ['nginx-web-server', '/wrk2-api/user/follow', 'nginx-web-server'],
  ['user-service', 'user_mmc_get_user_id_client', 'user-service'],
  ['user-service', 'get_user_id_server', 'social-graph-service'],
  ['user-service', 'get_user_id_server', 'social-graph-service'],
  ['user-service', 'user_mmc_get_user_id_client', 'user-service'],
  ['social-graph-service', 'mongo_update_client', 'social-graph-service'],
  ['social-graph-service',
   'social_graph_redis_update_client',
   'social-graph-service'],
  ['social-graph-service',
   'social_graph_mongo_update_client',
   'social-graph-service'],
  ['social-graph-service', 'follow_server', 'social-graph-service'],
  ['social-graph-service', 'follow_with_username_server', 'nginx-web-server']],
 [['user-service', 'user_mmc_get_user_id_client', 'user-service'],
  ['user-service', 'user_mmc_get_user_id_client', 'user-service'],
  ['user-service', 'get_user_id_server', 'social-graph-service'],

In [4]:
dataset[0]

[['nginx-web-server', 'Follow', 'nginx-web-server'],
 ['nginx-web-server', '/wrk2-api/user/follow', 'nginx-web-server'],
 ['user-service', 'user_mmc_get_user_id_client', 'user-service'],
 ['user-service', 'get_user_id_server', 'social-graph-service'],
 ['user-service', 'get_user_id_server', 'social-graph-service'],
 ['user-service', 'user_mmc_get_user_id_client', 'user-service'],
 ['social-graph-service', 'mongo_update_client', 'social-graph-service'],
 ['social-graph-service',
  'social_graph_redis_update_client',
  'social-graph-service'],
 ['social-graph-service',
  'social_graph_mongo_update_client',
  'social-graph-service'],
 ['social-graph-service', 'follow_server', 'social-graph-service'],
 ['social-graph-service', 'follow_with_username_server', 'nginx-web-server']]

In [5]:
dataset[0][0]

['nginx-web-server', 'Follow', 'nginx-web-server']

In [6]:
dataset[1]

[['user-service', 'user_mmc_get_user_id_client', 'user-service'],
 ['user-service', 'user_mmc_get_user_id_client', 'user-service'],
 ['user-service', 'get_user_id_server', 'social-graph-service'],
 ['user-service', 'get_user_id_server', 'social-graph-service'],
 ['nginx-web-server', '/wrk2-api/user/follow', 'nginx-web-server'],
 ['nginx-web-server', 'Follow', 'nginx-web-server'],
 ['social-graph-service', 'mongo_update_client', 'social-graph-service'],
 ['social-graph-service',
  'social_graph_redis_update_client',
  'social-graph-service'],
 ['social-graph-service', 'follow_with_username_server', 'nginx-web-server'],
 ['social-graph-service', 'follow_server', 'social-graph-service'],
 ['social-graph-service',
  'social_graph_mongo_update_client',
  'social-graph-service']]

In [7]:
dataset[2]

[['nginx-web-server', '/wrk2-api/user/follow', 'nginx-web-server'],
 ['nginx-web-server', 'Follow', 'nginx-web-server'],
 ['user-service', 'user_mmc_get_user_id_client', 'user-service'],
 ['user-service', 'get_user_id_server', 'social-graph-service'],
 ['user-service', 'user_mmc_get_user_id_client', 'user-service'],
 ['user-service', 'get_user_id_server', 'social-graph-service'],
 ['social-graph-service', 'mongo_update_client', 'social-graph-service'],
 ['social-graph-service',
  'social_graph_redis_update_client',
  'social-graph-service'],
 ['social-graph-service',
  'social_graph_mongo_update_client',
  'social-graph-service'],
 ['social-graph-service', 'follow_server', 'social-graph-service'],
 ['social-graph-service', 'follow_with_username_server', 'nginx-web-server']]

In [8]:
import asposecells
import jpype
jpype.startJVM()

from asposecells.api import Workbook, FileFormatType

def create_tsv(triplets, file_name):


    # Create Workbook object.
    workbook = Workbook(FileFormatType.TSV)

    # Access the first worksheet of the workbook.
    worksheet = workbook.getWorksheets().get(0)

    # Get the desired cell(s) of the worksheet and input the value into the cell(s).


    i = 1
    for elem in triplets: 
        worksheet.getCells().get("A"+str(i)).putValue(elem[0])
        worksheet.getCells().get("B"+str(i)).putValue(elem[1])
        worksheet.getCells().get("C"+str(i)).putValue(elem[2])
        i+=1


    # Save the workbook as TSV file.
    workbook.save(file_name)


    file = open(file_name,'r')  
    lines = file.readlines()  
    file.close()
    file = open(file_name,'w')  
    lines = lines[:-1]
    file.writelines(lines)
    file.close()

In [12]:
os.mkdir('train')
os.mkdir('validation')
os.mkdir('test')
for i in range(len(dataset)):
    folder = ""
    if i/len(dataset) < 0.6:
        folder = 'train'
    elif i/len(dataset) < 0.8:
        folder = 'validation'
    else:
        folder = 'test'
    create_tsv(dataset[i],os.getcwd() + '/' + folder + f"/20_social_graph_service_{i}.tsv")
jpype.shutdownJVM()
